In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import math

torch.manual_seed(0)
np.random.seed(0)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
#We first read the 2 data files
df1 = pd.read_csv('pirvision_office_dataset1.csv')
df2 = pd.read_csv('pirvision_office_dataset2.csv')

In [3]:
# #We first shuffle these 2 dataframes
# df1 = df1.sample(frac = 1, random_state=1).reset_index(drop=True)
# df2 = df2.sample(frac = 1, random_state=1).reset_index(drop=True)

#We now merge these 2 dataframes
df = pd.concat([df1, df2], ignore_index=True)

#We print the shapes of all datafmrames
print(df1.shape, df2.shape, df.shape)

#Displaying the merged dataframe
display(df.head())


(7651, 59) (7651, 59) (15302, 59)


,Date,Time,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,PIR_7,PIR_8,PIR_9,PIR_10,PIR_11,PIR_12,PIR_13,PIR_14,PIR_15,PIR_16,PIR_17,PIR_18,PIR_19,PIR_20,PIR_21,PIR_22,PIR_23,PIR_24,PIR_25,PIR_26,PIR_27,PIR_28,PIR_29,PIR_30,PIR_31,PIR_32,PIR_33,PIR_34,PIR_35,PIR_36,PIR_37,PIR_38,PIR_39,PIR_40,PIR_41,PIR_42,PIR_43,PIR_44,PIR_45,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55
0,2024-08-08,19:19:56,0,86,10269,10721,11156,11170,10931,10671,10395,10133,9885,9705,9538,9418,9469,9599,9817,9910,9890,10075,10231,10247,10271,10229,10272,10354,10449,10451,10419,10409,10336,10306,10356,10461,10456,10460,10467,10422,10303,9877,9308,9061,9299,9748,10209,10615,10975,11178,11197,11161,11096,10957,10839,10735,10590,10411,10329
1,2024-08-08,19:20:12,1,86,10364,10907,11299,11238,10867,10535,10173,9950,9856,9795,9714,9702,9792,9789,9915,9900,9944,9964,9971,10059,10161,10234,10285,10309,10384,10464,10450,10427,10366,10361,10452,10502,10444,10337,10250,10313,10211,9718,9236,9193,9609,10022,10431,10798,11055,11122,11145,11136,11108,11041,10824,10645,10493,10398,10357
2,2024-08-08,19:20:28,0,86,10329,10793,11197,11242,11052,10658,10288,9988,9819,9711,9659,9626,9726,9752,9835,9942,9925,9965,10110,10174,10140,10235,10303,10365,10366,10379,10375,10287,10310,10345,10373,10328,10387,10415,10491,10421,10432,9964,9368,9135,9287,9643,10184,10663,11016,11168,11204,11162,11109,11007,10867,10700,10533,10427,10265
3,2024-08-08,19:20:44,0,86,10169,10425,10822,11133,11136,10834,10520,10228,9986,9848,9643,9562,9591,9618,9718,9849,9857,10026,10150,10198,10261,10351,10425,10469,10374,10344,10303,10293,10294,10333,10353,10345,10354,10362,10375,10369,10319,10115,9603,9182,9125,9560,10161,10560,10883,11116,11273,11186,10984,10910,10807,10714,10651,10562,10463
4,2024-08-08,19:21:00,0,86,10320,10667,11104,11234,11129,10814,10453,10040,9733,9630,9578,9476,9596,9748,9755,9823,10004,10048,10202,10234,10255,10282,10298,10319,10315,10270,10334,10400,10428,10514,10529,10453,10374,10303,10298,10238,10246,9918,9399,9198,9422,9848,10225,10615,10860,11006,11257,11370,11173,10924,10816,10754,10588,10428,10407


In [4]:
# Ensure 'Date' and 'Time' are parsed correctly
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

# Extract temporal features
df['Hour'] = df['Datetime'].dt.hour
df['Minute'] = df['Datetime'].dt.minute
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Month'] = df['Datetime'].dt.month

df = df.drop(columns=['Date', 'Time', 'Datetime'])

meta_features = ['Hour', 'Minute', 'DayOfWeek', 'Month', 'Temperature_F']

In [5]:
df.head()

,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,PIR_7,PIR_8,PIR_9,PIR_10,PIR_11,PIR_12,PIR_13,PIR_14,PIR_15,PIR_16,PIR_17,PIR_18,PIR_19,PIR_20,PIR_21,PIR_22,PIR_23,PIR_24,PIR_25,PIR_26,PIR_27,PIR_28,PIR_29,PIR_30,PIR_31,PIR_32,PIR_33,PIR_34,PIR_35,PIR_36,PIR_37,PIR_38,PIR_39,PIR_40,PIR_41,PIR_42,PIR_43,PIR_44,PIR_45,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55,Hour,Minute,DayOfWeek,Month
0,0,86,10269,10721,11156,11170,10931,10671,10395,10133,9885,9705,9538,9418,9469,9599,9817,9910,9890,10075,10231,10247,10271,10229,10272,10354,10449,10451,10419,10409,10336,10306,10356,10461,10456,10460,10467,10422,10303,9877,9308,9061,9299,9748,10209,10615,10975,11178,11197,11161,11096,10957,10839,10735,10590,10411,10329,19,19,3,8
1,1,86,10364,10907,11299,11238,10867,10535,10173,9950,9856,9795,9714,9702,9792,9789,9915,9900,9944,9964,9971,10059,10161,10234,10285,10309,10384,10464,10450,10427,10366,10361,10452,10502,10444,10337,10250,10313,10211,9718,9236,9193,9609,10022,10431,10798,11055,11122,11145,11136,11108,11041,10824,10645,10493,10398,10357,19,20,3,8
2,0,86,10329,10793,11197,11242,11052,10658,10288,9988,9819,9711,9659,9626,9726,9752,9835,9942,9925,9965,10110,10174,10140,10235,10303,10365,10366,10379,10375,10287,10310,10345,10373,10328,10387,10415,10491,10421,10432,9964,9368,9135,9287,9643,10184,10663,11016,11168,11204,11162,11109,11007,10867,10700,10533,10427,10265,19,20,3,8
3,0,86,10169,10425,10822,11133,11136,10834,10520,10228,9986,9848,9643,9562,9591,9618,9718,9849,9857,10026,10150,10198,10261,10351,10425,10469,10374,10344,10303,10293,10294,10333,10353,10345,10354,10362,10375,10369,10319,10115,9603,9182,9125,9560,10161,10560,10883,11116,11273,11186,10984,10910,10807,10714,10651,10562,10463,19,20,3,8
4,0,86,10320,10667,11104,11234,11129,10814,10453,10040,9733,9630,9578,9476,9596,9748,9755,9823,10004,10048,10202,10234,10255,10282,10298,10319,10315,10270,10334,10400,10428,10514,10529,10453,10374,10303,10298,10238,10246,9918,9399,9198,9422,9848,10225,10615,10860,11006,11257,11370,11173,10924,10816,10754,10588,10428,10407,19,21,3,8


In [6]:
x = np.zeros((df.shape[0], 2+55+54+4))

for i in range(df.shape[0]):
  for j in range (0, 2):
    x[i,j] = df.iloc[i,j]
  for j in range(2, 55+54+2):
    if(j%2 == 0):
      x[i,j] = df.iloc[i,j//2 + 1]
    else:
      x[i,j] = (df.iloc[i,j//2 + 1] + df.iloc[i,j//2 + 2])/2

  for j in range(55+54+2, 55+54+2+4):
    x[i,j] = df.iloc[i,j-54]

In [7]:
x_df = pd.DataFrame(x)
x_df.columns = ['Label', 'Temperature_F'] + [f'PIR_{i}' for i in range(1, 110)] + ['Hour', 'Minute', 'DayOfWeek', 'Month']

In [8]:
x_df.head()

,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,PIR_7,PIR_8,PIR_9,PIR_10,PIR_11,PIR_12,PIR_13,PIR_14,PIR_15,PIR_16,PIR_17,PIR_18,PIR_19,PIR_20,PIR_21,PIR_22,PIR_23,PIR_24,PIR_25,PIR_26,PIR_27,PIR_28,PIR_29,PIR_30,PIR_31,PIR_32,PIR_33,PIR_34,PIR_35,PIR_36,PIR_37,PIR_38,PIR_39,PIR_40,PIR_41,PIR_42,PIR_43,PIR_44,PIR_45,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55,PIR_56,PIR_57,PIR_58,PIR_59,PIR_60,PIR_61,PIR_62,PIR_63,PIR_64,PIR_65,PIR_66,PIR_67,PIR_68,PIR_69,PIR_70,PIR_71,PIR_72,PIR_73,PIR_74,PIR_75,PIR_76,PIR_77,PIR_78,PIR_79,PIR_80,PIR_81,PIR_82,PIR_83,PIR_84,PIR_85,PIR_86,PIR_87,PIR_88,PIR_89,PIR_90,PIR_91,PIR_92,PIR_93,PIR_94,PIR_95,PIR_96,PIR_97,PIR_98,PIR_99,PIR_100,PIR_101,PIR_102,PIR_103,PIR_104,PIR_105,PIR_106,PIR_107,PIR_108,PIR_109,Hour,Minute,DayOfWeek,Month
0,0.0,86.0,10269.0,10495.0,10721.0,10938.5,11156.0,11163.0,11170.0,11050.5,10931.0,10801.0,10671.0,10533.0,10395.0,10264.0,10133.0,10009.0,9885.0,9795.0,9705.0,9621.5,9538.0,9478.0,9418.0,9443.5,9469.0,9534.0,9599.0,9708.0,9817.0,9863.5,9910.0,9900.0,9890.0,9982.5,10075.0,10153.0,10231.0,10239.0,10247.0,10259.0,10271.0,10250.0,10229.0,10250.5,10272.0,10313.0,10354.0,10401.5,10449.0,10450.0,10451.0,10435.0,10419.0,10414.0,10409.0,10372.5,10336.0,10321.0,10306.0,10331.0,10356.0,10408.5,10461.0,10458.5,10456.0,10458.0,10460.0,10463.5,10467.0,10444.5,10422.0,10362.5,10303.0,10090.0,9877.0,9592.5,9308.0,9184.5,9061.0,9180.0,9299.0,9523.5,9748.0,9978.5,10209.0,10412.0,10615.0,10795.0,10975.0,11076.5,11178.0,11187.5,11197.0,11179.0,11161.0,11128.5,11096.0,11026.5,10957.0,10898.0,10839.0,10787.0,10735.0,10662.5,10590.0,10500.5,10411.0,10370.0,10329.0,19.0,19.0,3.0,8.0
1,1.0,86.0,10364.0,10635.5,10907.0,11103.0,11299.0,11268.5,11238.0,11052.5,10867.0,10701.0,10535.0,10354.0,10173.0,10061.5,9950.0,9903.0,9856.0,9825.5,9795.0,9754.5,9714.0,9708.0,9702.0,9747.0,9792.0,9790.5,9789.0,9852.0,9915.0,9907.5,9900.0,9922.0,9944.0,9954.0,9964.0,9967.5,9971.0,10015.0,10059.0,10110.0,10161.0,10197.5,10234.0,10259.5,10285.0,10297.0,10309.0,10346.5,10384.0,10424.0,10464.0,10457.0,10450.0,10438.5,10427.0,10396.5,10366.0,10363.5,10361.0,10406.5,10452.0,10477.0,10502.0,10473.0,10444.0,10390.5,10337.0,10293.5,10250.0,10281.5,10313.0,10262.0,10211.0,9964.5,9718.0,9477.0,9236.0,9214.5,9193.0,9401.0,9609.0,9815.5,10022.0,10226.5,10431.0,10614.5,10798.0,10926.5,11055.0,11088.5,11122.0,11133.5,11145.0,11140.5,11136.0,11122.0,11108.0,11074.5,11041.0,10932.5,10824.0,10734.5,10645.0,10569.0,10493.0,10445.5,10398.0,10377.5,10357.0,19.0,20.0,3.0,8.0
2,0.0,86.0,10329.0,10561.0,10793.0,10995.0,11197.0,11219.5,11242.0,11147.0,11052.0,10855.0,10658.0,10473.0,10288.0,10138.0,9988.0,9903.5,9819.0,9765.0,9711.0,9685.0,9659.0,9642.5,9626.0,9676.0,9726.0,9739.0,9752.0,9793.5,9835.0,9888.5,9942.0,9933.5,9925.0,9945.0,9965.0,10037.5,10110.0,10142.0,10174.0,10157.0,10140.0,10187.5,10235.0,10269.0,10303.0,10334.0,10365.0,10365.5,10366.0,10372.5,10379.0,10377.0,10375.0,10331.0,10287.0,10298.5,10310.0,10327.5,10345.0,10359.0,10373.0,10350.5,10328.0,10357.5,10387.0,10401.0,10415.0,10453.0,10491.0,10456.0,10421.0,10426.5,10432.0,10198.0,9964.0,9666.0,9368.0,9251.5,9135.0,9211.0,9287.0,9465.0,9643.0,9913.5,10184.0,10423.5,10663.0,10839.5,11016.0,11092.0,11168.0,11186.0,11204.0,11183.0,11162.0,11135.5,11109.0,11058.0,11007.0,10937.0,10867.0,10783.5,10700.0,10616.5,10533.0,10480.0,10427.0,10346.0,10265.0,19.0,20.0,3.0,8.0
3,0.0,86.0,10169.0,10297.0,10425.0,10623.5,10822.0,10977.5,11133.0,11134.5,11136.0,10985.0,10834.0,10677.0,10520.0,10374.0,10228.0,10107.0,9986.0,9917.0,9848.0,9745.5,9643.0,9602.5,9562.0,9576.5,9591.0,9604.5,9618.0,9668.0,9718.0,9783.5,9849.0,9853.0,9857.0,9941.5,10026.0,10088.0,10150.0,10174.0,10198.0,10229.5,10261.0,10306.0,10351.0,10388.0,10425.0,10447.0,10469.0,10421.5,10374.0,10359.0,10344.0,10323.5,10303.0,10298.0,10293.0,10293.5,10294.0,10313.5,10333.0,10343.0,10353.0,10349.0,10345.0,10349.5,10354.0,10358.0,10362.0,10368.5,10375.0,10372.0,10369.0,10344.0,10319.

In [9]:
df = x_df

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler


X = df.drop("Label", axis=1)
y = df["Label"]

#SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# #Random Oversampler
# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_resample(X, y)

df = pd.concat([X_resampled, y_resampled], axis=1)

In [11]:
df.head()

,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,PIR_7,PIR_8,PIR_9,PIR_10,PIR_11,PIR_12,PIR_13,PIR_14,PIR_15,PIR_16,PIR_17,PIR_18,PIR_19,PIR_20,PIR_21,PIR_22,PIR_23,PIR_24,PIR_25,PIR_26,PIR_27,PIR_28,PIR_29,PIR_30,PIR_31,PIR_32,PIR_33,PIR_34,PIR_35,PIR_36,PIR_37,PIR_38,PIR_39,PIR_40,PIR_41,PIR_42,PIR_43,PIR_44,PIR_45,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55,PIR_56,PIR_57,PIR_58,PIR_59,PIR_60,PIR_61,PIR_62,PIR_63,PIR_64,PIR_65,PIR_66,PIR_67,PIR_68,PIR_69,PIR_70,PIR_71,PIR_72,PIR_73,PIR_74,PIR_75,PIR_76,PIR_77,PIR_78,PIR_79,PIR_80,PIR_81,PIR_82,PIR_83,PIR_84,PIR_85,PIR_86,PIR_87,PIR_88,PIR_89,PIR_90,PIR_91,PIR_92,PIR_93,PIR_94,PIR_95,PIR_96,PIR_97,PIR_98,PIR_99,PIR_100,PIR_101,PIR_102,PIR_103,PIR_104,PIR_105,PIR_106,PIR_107,PIR_108,PIR_109,Hour,Minute,DayOfWeek,Month,Label
0,86.0,10269.0,10495.0,10721.0,10938.5,11156.0,11163.0,11170.0,11050.5,10931.0,10801.0,10671.0,10533.0,10395.0,10264.0,10133.0,10009.0,9885.0,9795.0,9705.0,9621.5,9538.0,9478.0,9418.0,9443.5,9469.0,9534.0,9599.0,9708.0,9817.0,9863.5,9910.0,9900.0,9890.0,9982.5,10075.0,10153.0,10231.0,10239.0,10247.0,10259.0,10271.0,10250.0,10229.0,10250.5,10272.0,10313.0,10354.0,10401.5,10449.0,10450.0,10451.0,10435.0,10419.0,10414.0,10409.0,10372.5,10336.0,10321.0,10306.0,10331.0,10356.0,10408.5,10461.0,10458.5,10456.0,10458.0,10460.0,10463.5,10467.0,10444.5,10422.0,10362.5,10303.0,10090.0,9877.0,9592.5,9308.0,9184.5,9061.0,9180.0,9299.0,9523.5,9748.0,9978.5,10209.0,10412.0,10615.0,10795.0,10975.0,11076.5,11178.0,11187.5,11197.0,11179.0,11161.0,11128.5,11096.0,11026.5,10957.0,10898.0,10839.0,10787.0,10735.0,10662.5,10590.0,10500.5,10411.0,10370.0,10329.0,19.0,19.0,3.0,8.0,0.0
1,86.0,10364.0,10635.5,10907.0,11103.0,11299.0,11268.5,11238.0,11052.5,10867.0,10701.0,10535.0,10354.0,10173.0,10061.5,9950.0,9903.0,9856.0,9825.5,9795.0,9754.5,9714.0,9708.0,9702.0,9747.0,9792.0,9790.5,9789.0,9852.0,9915.0,9907.5,9900.0,9922.0,9944.0,9954.0,9964.0,9967.5,9971.0,10015.0,10059.0,10110.0,10161.0,10197.5,10234.0,10259.5,10285.0,10297.0,10309.0,10346.5,10384.0,10424.0,10464.0,10457.0,10450.0,10438.5,10427.0,10396.5,10366.0,10363.5,10361.0,10406.5,10452.0,10477.0,10502.0,10473.0,10444.0,10390.5,10337.0,10293.5,10250.0,10281.5,10313.0,10262.0,10211.0,9964.5,9718.0,9477.0,9236.0,9214.5,9193.0,9401.0,9609.0,9815.5,10022.0,10226.5,10431.0,10614.5,10798.0,10926.5,11055.0,11088.5,11122.0,11133.5,11145.0,11140.5,11136.0,11122.0,11108.0,11074.5,11041.0,10932.5,10824.0,10734.5,10645.0,10569.0,10493.0,10445.5,10398.0,10377.5,10357.0,19.0,20.0,3.0,8.0,1.0
2,86.0,10329.0,10561.0,10793.0,10995.0,11197.0,11219.5,11242.0,11147.0,11052.0,10855.0,10658.0,10473.0,10288.0,10138.0,9988.0,9903.5,9819.0,9765.0,9711.0,9685.0,9659.0,9642.5,9626.0,9676.0,9726.0,9739.0,9752.0,9793.5,9835.0,9888.5,9942.0,9933.5,9925.0,9945.0,9965.0,10037.5,10110.0,10142.0,10174.0,10157.0,10140.0,10187.5,10235.0,10269.0,10303.0,10334.0,10365.0,10365.5,10366.0,10372.5,10379.0,10377.0,10375.0,10331.0,10287.0,10298.5,10310.0,10327.5,10345.0,10359.0,10373.0,10350.5,10328.0,10357.5,10387.0,10401.0,10415.0,10453.0,10491.0,10456.0,10421.0,10426.5,10432.0,10198.0,9964.0,9666.0,9368.0,9251.5,9135.0,9211.0,9287.0,9465.0,9643.0,9913.5,10184.0,10423.5,10663.0,10839.5,11016.0,11092.0,11168.0,11186.0,11204.0,11183.0,11162.0,11135.5,11109.0,11058.0,11007.0,10937.0,10867.0,10783.5,10700.0,10616.5,10533.0,10480.0,10427.0,10346.0,10265.0,19.0,20.0,3.0,8.0,0.0
3,86.0,10169.0,10297.0,10425.0,10623.5,10822.0,10977.5,11133.0,11134.5,11136.0,10985.0,10834.0,10677.0,10520.0,10374.0,10228.0,10107.0,9986.0,9917.0,9848.0,9745.5,9643.0,9602.5,9562.0,9576.5,9591.0,9604.5,9618.0,9668.0,9718.0,9783.5,9849.0,9853.0,9857.0,9941.5,10026.0,10088.0,10150.0,10174.0,10198.0,10229.5,10261.0,10306.0,10351.0,10388.0,10425.0,10447.0,10469.0,10421.5,10374.0,10359.0,10344.0,10323.5,10303.0,10298.0,10293.0,10293.5,10294.0,10313.5,10333.0,10343.0,10353.0,10349.0,10345.0,10349.5,10354.0,10358.0,10362.0,10368.5,10375.0,10372.0,10369.0,10344.0,10319.0,10

In [12]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]


class HybridTransformer(nn.Module):
    def __init__(self, input_size=1, meta_input_size=5, seq_len=109, d_model=64, nhead=4, num_layers=9, num_classes=3):
        super().__init__()
        self.input_proj = nn.Linear(input_size, d_model)  # PIR projection
        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=128, dropout=0.1, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.meta_mlp = nn.Sequential(
            nn.Linear(meta_input_size, 32),
            nn.ReLU(),
            nn.Linear(32, d_model),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(d_model * 2, 64),  # Transformer + Metadata
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x_seq, x_meta):
        # x_seq: (batch_size, seq_len, 1)
        x_seq = self.input_proj(x_seq)         # (B, seq_len, d_model)
        x_seq = self.pos_encoder(x_seq)        # Add positional encoding
        x_seq = self.transformer_encoder(x_seq)  # (B, seq_len, d_model)
        x_seq = x_seq.mean(dim=1)              # Mean pooling over time

        # x_meta: (batch_size, meta_input_size)
        x_meta = self.meta_mlp(x_meta)         # → (B, d_model)

        combined = torch.cat([x_seq, x_meta], dim=1)  # (B, 2*d_model)
        return self.classifier(combined)              # (B, num_classes)


In [13]:
print("Class distribution in df:")
print(df["Label"].value_counts())

Class distribution in df:
Label
0.0    12494
1.0    12494
3.0    12494
Name: count, dtype: int64


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

from torch.utils.data import Dataset
import torch
from sklearn.metrics import classification_report, f1_score

class HybridTimeSeriesDataset(Dataset):
    def __init__(self, X_seq, X_meta, y):
        self.X_seq = X_seq
        self.X_meta = X_meta
        self.y = y

    def __len__(self):
        return len(self.X_seq)

    def __getitem__(self, idx):
        sequence = self.X_seq[idx]                # shape: (55,)
        meta = self.X_meta[idx]                   # shape: (9,)
        label = self.y[idx]

        sequence_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(1)  # (55, 1)
        meta_tensor = torch.tensor(meta, dtype=torch.float32)                       # (9,)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return sequence_tensor, meta_tensor, label_tensor

pir_columns = [f'PIR_{i}' for i in range(1, 110)]
X_seq = df[pir_columns].values

# Metadata features
X_meta = df[meta_features].values

# Labels (remapped as before)
label_map = {0: 0, 1: 1, 3: 2}
y_raw = df["Label"].values
y = np.array([label_map[label] for label in y_raw])
kf = KFold(n_splits=5, shuffle=True, random_state=0)

input_size = 1
model = HybridTransformer(input_size=input_size)

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

# Use in CrossEntropyLoss
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = optim.Adam(model.parameters(), lr=0.001)

nb_epochs = 8
batch_size = 64  # you can adjust this based on memory

for fold, (train_i, test_i) in enumerate(kf.split(X_seq), 1):
    print(f"\nFold {fold}")
    if fold == 2:
        break

    X_seq_train, X_seq_test = X_seq[train_i], X_seq[test_i]
    X_meta_train, X_meta_test = X_meta[train_i], X_meta[test_i]
    y_train, y_test = y[train_i], y[test_i]



    print("Train label distribution:", np.bincount(y_train))
    print("Test label distribution:", np.bincount(y_test))

    train_dataset = HybridTimeSeriesDataset(X_seq_train, X_meta_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(nb_epochs):
        total_loss = 0.0
        model.train()
        for sequences, metas, labels in train_loader:
            output = model(sequences, metas)  # Forward pass
            loss = criterion(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * sequences.size(0)

        avg_loss = total_loss / len(train_dataset)
        print(f"epoch {epoch+1}, loss: {avg_loss:.4f}")

    # --- Evaluation ---
    model.eval()
    y_true = []
    y_pred = []
    correct = 0
    total = 0
    class_counts = {0: 0, 1: 0, 2: 0}
    class_correct = {0: 0, 1: 0, 2: 0}

    with torch.no_grad():
        for i in range(X_seq_test.shape[0]):
            sequence_tensor = torch.tensor(X_seq_test[i], dtype=torch.float32).unsqueeze(0).unsqueeze(2)
            meta_tensor = torch.tensor(X_meta_test[i], dtype=torch.float32).unsqueeze(0)
            label = y_test[i]

            output = model(sequence_tensor, meta_tensor)
            predicted_class = torch.argmax(output, dim=1).item()

            y_true.append(label)
            y_pred.append(predicted_class)

            class_counts[label] += 1
            if predicted_class == label:
                correct += 1
                class_correct[label] += 1
            total += 1

    accuracy = correct / total
    print(f"\nfold {fold} test accuracy: {accuracy:.4f}")

    print("\nPer-class accuracy:")
    for cls in class_counts:
        total_cls = class_counts[cls]
        correct_cls = class_correct[cls]
        acc_cls = correct_cls / total_cls if total_cls > 0 else 0.0
        print(f"  Class {cls}: {correct_cls}/{total_cls} correct ({acc_cls * 100:.2f}%)")

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=4))

    macro_f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Macro F1-Score: {macro_f1:.4f}")


Fold 1
Train label distribution: [ 9996  9940 10049]
Test label distribution: [2498 2554 2445]
epoch 1, loss: 0.2596
epoch 2, loss: 0.1447
epoch 3, loss: 0.1426
epoch 4, loss: 0.1359
epoch 5, loss: 0.1269
epoch 6, loss: 0.1157
epoch 7, loss: 0.1062
epoch 8, loss: 0.1016

fold 1 test accuracy: 0.9518

Per-class accuracy:
  Class 0: 2274/2498 correct (91.03%)
  Class 1: 2417/2554 correct (94.64%)
  Class 2: 2445/2445 correct (100.00%)

Classification Report:
              precision    recall  f1-score   support

           0     0.9432    0.9103    0.9265      2498
           1     0.9152    0.9464    0.9305      2554
           2     1.0000    1.0000    1.0000      2445

    accuracy                         0.9518      7497
   macro avg     0.9528    0.9522    0.9523      7497
weighted avg     0.9522    0.9518    0.9518      7497

Macro F1-Score: 0.9523

Fold 2
